#Anime Planet // Vulpix // CS109 Final Project
Tuongvan Le, Tiffany Lee, Kelwen Peng, Dylan Tan

##Scraping Data
###Reviews and Users

In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import re
import string
import json
import pickle
from collections import defaultdict

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier than the built-in urllib libraries.
import requests
from urllib2 import urlopen

In [41]:
test1 = open("../data/anime_pages.p",'rb') 
anime = pickle.load(test1)  
test1.close()

In [42]:
root = 'http://www.anime-planet.com'

all_anime = []
for page in anime:
    soup = BeautifulSoup(page.text, "html.parser")
    animelist = soup.find('ul',attrs={"class": "cardDeck pure-g cd-narrow"}).find_all("li")
    for ani in animelist:
        all_anime.append(root + ani.find('a').get('href'))

In [95]:
%%time
anime_reviews = defaultdict(list)
stop = 0
for i in xrange(len(all_anime)):
    if i>=stop:
        print i
        stop+=500
    review = requests.get(all_anime[i]+"/reviews")
    soup1 = BeautifulSoup(review.text, "html.parser")
    pages = soup1.find('section',attrs={'pure-g'})
    if pages:
        pages = pages.find('div',attrs='pure-1 md-4-5')
    if pages:
        pages = pages.find('div',attrs='pagination aligncenter')
        if pages:
            all_li = pages.ul.find_all('li')
            maxpage = int(all_li[len(all_li)-2].a.contents[0])
        else:
            maxpage=1
    else:
        maxpage=1
    for j in xrange(1,maxpage+1):
        anime_reviews[all_anime[i].split('/')[-1]].append(requests.get(all_anime[i]+"/reviews?page="+str(j)))
        time.sleep(1)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
CPU times: user 5min 47s, sys: 4min 43s, total: 10min 30s
Wall time: 4h 3min 37s


In [168]:
#pickle.dump(anime_reviews, open("../data/anime_reviews1.p", "wb"))

test3 = open("../data/anime_reviews1.p",'rb') 
anime_reviews = pickle.load(test3)  
test3.close()

In [215]:
reviewsdict = {}
for anime in anime_reviews.keys():
    animedict = {anime:{}}
    for a in range(len(anime_reviews[anime])):
        soup = BeautifulSoup(anime_reviews[anime][a].text, "html.parser")
        body = soup.find('div',attrs={"class": "pure-1 md-4-5"})
        if body is not None:
            sections = body.find_all('section', attrs={'class':'pure-g'})
            names = [i.find('a',attrs={'itemprop':'author'}).contents[0] for i in sections if i.find('a',attrs={'itemprop':'author'}) is not None]
            scores = [i.find_all('div',attrs={'class':'pure-1-5'}) for i in sections if i.find_all('div',attrs={'class':'pure-1-5'}) != []]
            story = [i[0].contents[0] for i in scores]
            animation = [i[1].contents[0] for i in scores]
            sound = [i[2].contents[0] for i in scores]
            characters = [i[3].contents[0] for i in scores]
            overall = [i[4].contents[0] for i in scores]
            reviews = [i.contents[1].string for i in body.find_all('section', attrs={'class':'userContent readMore'})]
            anime_name = soup.find('h1',attrs={"itemprop": "name"}).contents*len(names)
            temp = dict(zip(names,zip(story,animation,sound,characters,overall,reviews,anime_name)))
            animedict[anime].update(temp)
    reviewsdict.update(animedict)


In [222]:
with open('../data/all_reviews.json', 'w') as fp:
      json.dump(reviewsdict, fp)

with open('../data/all_reviews.json','r') as data_file:    
    reviewsdict = json.load(data_file)

In [216]:
reviewlist = []
for i in reviewsdict.keys():
    reviewlist+=reviewsdict[i].keys()
users = list(set(reviewlist))

In [217]:
len(users),len(reviewlist)

(4462, 17194)

In [218]:
newlist = []

for key in reviewsdict:
    for user in reviewsdict[key]:
        story = reviewsdict[key][user][0]
        animation = reviewsdict[key][user][1]
        sound = reviewsdict[key][user][2]
        character = reviewsdict[key][user][3]
        overall = reviewsdict[key][user][4]
        reviews = reviewsdict[key][user][5]
        anime_name = reviewsdict[key][user][6]
        temp = {"name":user,'anime':key,'story':story,'animation':animation,'sound':sound,'character':character,'overall':overall,'reviews':reviews,'anime_name':anime_name}
        newlist.append(temp)

In [221]:
newdata = pd.DataFrame(newlist)
newdata.shape

(17194, 9)

In [223]:
newdata.to_csv('../data/all_reviews.csv',sep='\t',encoding='utf-8')

In [36]:
#newdata.loc[:,['anime','anime_name']].to_csv('../data/anime_name.csv',sep='\t',encoding='utf-8')

#### Getting User Profile

In [8]:
%%time
root = 'http://www.anime-planet.com/users/'
user_profiles = {}
for i in xrange(len(users)):
    user = requests.get(root+users[i])
    soup = BeautifulSoup(user.text, "html.parser")
    user_profiles[users[i]]=soup
    time.sleep(1)

CPU times: user 3min 17s, sys: 26.8 s, total: 3min 44s
Wall time: 1h 46min 45s


In [91]:
each_user_info = {}

for user in user_profiles.keys():
    test = user_profiles[user]
    user_info = {}

    info = test.find('div',attrs={'class':'pure-u-3-5 pure-u-md-4-5'})
    name = info.find('h2',attrs={'id':'profileName'}).find_all('a')[1].contents[0]

    stats = info.find('ul',attrs={'class':'userStats'}).findAll('li')
    place = ""
    age_sex = ""
    join = ""

    # find place, age, sex if exist
    for li in stats:
        if li.findAll(attrs={"class" : "fa fa-home"}):
            place = li.contents[1]
        elif li.findAll(attrs={"class" : "fa fa-calendar"}):
            join = li.contents[1]
        elif li.findAll(attrs={"class" : "fa fa-user"}):
            age_sex = li.contents[2].replace('\n','')

    each_user_info[name] = {"place":place,"age_sex":age_sex,"join":join}

In [2]:
# with open('../data/all_users.json', 'w') as fp:
#        json.dump(each_user_info, fp)

with open('../data/all_users.json','r') as data_file:    
    each_user_info = json.load(data_file)

In [3]:
userdf = pd.DataFrame(each_user_info).T

,age_sex,join,place
00081,26,"Joined Jan 16, 2012",Death Star
0Marcandre,20 / M,"Joined Mar 31, 2013",
0oDaano0,?,"Joined Apr 9, 2013",
0utl4w,?,"Joined Mar 31, 2010",somewhere
11bowman,21 / M,"Joined Jun 19, 2010","Amsterdam, Netherlands"


In [4]:
userdf.to_csv('../data/all_users.csv',sep='\t',encoding='utf-8')

###Anime Data

In [ ]:
# import pip

# def install(package):
#    pip.main(['install', package])

# install('python-mal')

# import myanimelist.session
# #username="teamvulpix", password="2015harvardcs109"
# session = myanimelist.session.Session()
# session.login()
# bebop = session.anime(20)

#### Scraping the search bar of www.anime-planet.com

In [ ]:
#48 animes per page with 152 pages = ~7296
anime = []
for i in xrange(1,153):
    anime.append(requests.get('http://www.anime-planet.com/anime/all?sort=title&order=asc&page='+str(i)))
    time.sleep(1)

#### Finding all anime links

In [ ]:
root = 'http://www.anime-planet.com'

all_anime = []
for page in anime:
    soup = BeautifulSoup(page.text, "html.parser")
    animelist = soup.find('ul',attrs={"class": "cardDeck pure-g cd-narrow"}).find_all("li")
    for ani in animelist:
        all_anime.append(root + ani.find('a').get('href'))

#### Requesting each anime page information

In [ ]:
each_anime = []

for i in xrange(len(all_anime)):
    each_anime.append(requests.get(all_anime[i]))
    time.sleep(1)

####Getting info from each anime

In [ ]:
each_anime_info = {}

for anim in each_anime:
    anime_info = {}
    soup1 = BeautifulSoup(anim.text, "html.parser")
    name = soup1.find('h1',attrs={"itemprop": "name"}).contents[0]
    altname = soup1.find('h2',attrs={"class": "aka"})

    infos = soup1.find_all('div',attrs={"class": "pure-u-1 pure-u-md-1-5"})
    atype = infos[0]
    studio = infos[1].a
    date = infos[2].li
    date_pub = infos[2].find('span',attrs={'itemprop':'datePublished'})
    rating = infos[3].div.span
    rank = infos[4]
    description = soup1.find('div',attrs={"itemprop": "description"})
    
    #get categories
    categories_table = soup1.find('div',attrs={"class": "categories"})
    if categories_table is not None:
        anime_info['categories'] = [i.a.contents[0] for i in categories_table.ul]
    
    #get related media
    related_table = soup1.find('table',attrs={"class": "pure-table pure-table-striped noHeader"})
    if related_table is not None:
        anime_info['related'] = [row.td.a.contents[0] for row in related_table.find_all('tr')]
    
    anime_info['name'] = name
    if altname is not None:
        anime_info['altname'] = altname.contents[0]
    else:
        anime_info['altname'] = ''
    if atype is not None:
        anime_info['atype'] = atype.contents[0]
    else:
        anime_info['atype'] = ''
    if studio is not None:
        anime_info['studio'] = studio.contents[0]
    else:
        anime_info['studio'] = ''
    if date is not None:
        anime_info['date'] = date.a.contents[0]
    else:
        anime_info['date'] = ''
    if date_pub is not None:
        anime_info['date_pub'] = date_pub.a.contents[0]
    else:
        anime_info['date'] = ''
    if rating is not None:
        anime_info['rating'] = rating.contents[0]
    else:
        anime_info['rating'] = ''
    if rank is not None:
        anime_info['rank'] = rank.contents[0]
    else:
        anime_info['rank'] = ''
    if description is not None :
        if description.p is not None:
            anime_info['description'] = description.p.contents[0].string
        else:
                anime_info['description'] = ''
    else:
        anime_info['description'] = ''
    
    if name in each_anime_info.keys():
        name+='_2'
        
    each_anime_info[name] = anime_info

In [ ]:
#  with open('each_anime_info.json', 'w') as fp:
#      json.dump(each_anime_info, fp)

with open('../data/each_anime_info.json','r') as data_file:    
    each_anime_info = json.load(data_file)
    
anime_info = pd.DataFrame(each_anime_info).T
anime_info.head()

In [ ]:
#anime_info.to_csv('../data/anime_info.csv',sep='\t',encoding='utf-8')
anime_info = pd.read_csv('../data/anime_info.csv',sep='\t',index_col=0)
anime_info.head()